# 任務
快速做出一個可以自動參考不同文件的聊天型問答機器人，基本上就是智能客服。

In [1]:
from operator import itemgetter
from langchain.document_loaders import NotionDirectoryLoader
from langchain.text_splitter import MarkdownTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Qdrant
from langchain.memory import ConversationBufferMemory
from langchain.schema import StrOutputParser, BaseMessage, HumanMessage, AIMessage
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.prompts import PromptTemplate, ChatPromptTemplate

from langchain_setup import ChatOpenAI, tracing_v2_enabled_if_api_key_set

# 1. 聊天紀錄

In [2]:
memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

In [3]:
memory.load_memory_variables(12345678) # input is just for compatability and is ignored

{'history': []}

In [4]:
memory.save_context(inputs={'question': "學長給的水可不可以喝?"}, outputs={"answer": "先用打火機試看看能不能點燃。"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='學長給的水可不可以喝?'),
  AIMessage(content='先用打火機試看看能不能點燃。')]}

# 2. 問題改寫

In [5]:
def format_chat_history(chat_history: list[BaseMessage]) -> str:
    buffer = ""
    for message in chat_history:
        if isinstance(message, HumanMessage):
            buffer += f"人類: {message.content}\n"
        elif isinstance(message, AIMessage):
            buffer += f"助理: {message.content}\n"
    return buffer

chat_history = [
    HumanMessage(content="哈囉"), 
    AIMessage(content="你好"), 
    HumanMessage(content="你生氣了嗎?"),
    AIMessage(content="沒有生氣呦")
]
formatted_chat_history = format_chat_history(chat_history)
print(formatted_chat_history)

人類: 哈囉
助理: 你好
人類: 你生氣了嗎?
助理: 沒有生氣呦



In [6]:
condense_question_template = """根據以下的對話紀錄和接續的問題，將該接續的問題改寫為一個獨立的問題。

聊天紀錄:
{chat_history}
接續的問題: {question}
獨立的問題:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(condense_question_template)

print(CONDENSE_QUESTION_PROMPT.format(question="等一下要吃什麼?",chat_history=formatted_chat_history))

根據以下的對話紀錄和接續的問題，將該接續的問題改寫為一個獨立的問題。

聊天紀錄:
人類: 哈囉
助理: 你好
人類: 你生氣了嗎?
助理: 沒有生氣呦

接續的問題: 等一下要吃什麼?
獨立的問題:


# 3. 抽取相關文件

## 3.1 Loader (讀取器)

In [7]:
loader = NotionDirectoryLoader(path="../data/notion")
documents = loader.load()
documents

[Document(page_content='今天來介紹日本最具代表性的軟體開發手法\n它的名子為 **隕石落下型開發**。\n\n# 第一節\n\n通常的**瀑布式開發**是像下面這樣的形式:\n| 步驟 | 內容     | 負責人     |\n| ---- | -------- | ---------- |\n| 1    | 要件定義 | Producer   |\n| 2    | 基本設計 | Director   |\n| 3    | 詳細設計 | Planner    |\n| 4    | 實裝     | Programmer |\n\n\n而**隕石式開發**是像下面這樣子的形式：\n|     | 步驟 | 內容     | 負責人     |\n| --- | ---- | -------- | ---------- |\n| 神  | 1    | 要件定義 | Producer   |\n| 神  | 2    | 基本設計 | Director   |\n| 神  | 3    | 詳細設計 | Planner    |\n| 神  | 4    | 實裝     | Programmer |\n\n然後就會這樣（全部都被隕石砸到爆炸）：\n\n💥要件定義💥Producer💥基本設計💥Director💥詳細設計💥Planner💥實裝💥Programmer\n\n這是敏捷式開法守法的循環\n\n[要件定義->基本設計->詳細設計->實裝]->[要件定義->基本設計->詳細設計->實裝]->\n\n但在神的面前都是無力的（全部都被隕石砸到爆炸）\n\n[要件💥定義-💥>基本設計💥->詳細設💥計->實💥裝]<->[要件定💥義->基本設💥計->詳💥細設💥計->💥實裝]->\n\n在神的一聲令下全部都會**崩壞**，\n\n而人民會努力地**重建**，\n\n這就是 -- 隕石落下式開發法。\n\n# 第二節\n\n全部的計畫表都是由**天界**的一己之願來訂定，這被稱為**默示錄**\n\n雖然在軟體工程中回饋是個重要的因素\n\n但回饋**不會傳達**到神的耳裡\n\n只是我們可以獻上我們的**祈禱**，這些祈禱在**非常稀有**的情況下能上達天聽。\n\n神會以**各種姿態**出現，有從外部來的，也有從內部來的。\n\n或者也可能是

## 3.2 Splitter (切分器)

In [8]:
splitter = MarkdownTextSplitter(chunk_size=500, chunk_overlap=100)
documents = splitter.split_documents(documents=documents)
documents

[Document(page_content='今天來介紹日本最具代表性的軟體開發手法\n它的名子為 **隕石落下型開發**。\n\n# 第一節\n\n通常的**瀑布式開發**是像下面這樣的形式:\n| 步驟 | 內容     | 負責人     |\n| ---- | -------- | ---------- |\n| 1    | 要件定義 | Producer   |\n| 2    | 基本設計 | Director   |\n| 3    | 詳細設計 | Planner    |\n| 4    | 實裝     | Programmer |\n\n\n而**隕石式開發**是像下面這樣子的形式：\n|     | 步驟 | 內容     | 負責人     |\n| --- | ---- | -------- | ---------- |\n| 神  | 1    | 要件定義 | Producer   |\n| 神  | 2    | 基本設計 | Director   |\n| 神  | 3    | 詳細設計 | Planner    |\n| 神  | 4    | 實裝     | Programmer |\n\n然後就會這樣（全部都被隕石砸到爆炸）：', metadata={'source': '..\\data\\notion\\隕石落下式開發法.md'}),
 Document(page_content='然後就會這樣（全部都被隕石砸到爆炸）：\n\n💥要件定義💥Producer💥基本設計💥Director💥詳細設計💥Planner💥實裝💥Programmer\n\n這是敏捷式開法守法的循環\n\n[要件定義->基本設計->詳細設計->實裝]->[要件定義->基本設計->詳細設計->實裝]->\n\n但在神的面前都是無力的（全部都被隕石砸到爆炸）\n\n[要件💥定義-💥>基本設計💥->詳細設💥計->實💥裝]<->[要件定💥義->基本設💥計->詳💥細設💥計->💥實裝]->\n\n在神的一聲令下全部都會**崩壞**，\n\n而人民會努力地**重建**，\n\n這就是 -- 隕石落下式開發法。\n\n# 第二節\n\n全部的計畫表都是由**天界**的一己之願來訂定，這被稱為**默示錄**\n\n雖然在軟體工程中回饋是個重要的因素\n\n但回饋

## 3.3 VectorStore (向量存儲)

In [9]:
embeder = OpenAIEmbeddings(model="text-embedding-ada-002")
vectorstore = Qdrant.from_documents(documents=documents, embedding=embeder, location=":memory:")

## 3.4 Retriever (抽取器)

In [10]:
retriever = vectorstore.as_retriever(search_kwargs=dict(k=2)) # top k similar documents
retriever.get_relevant_documents("軟體開發中的隕石是什麼？")

[Document(page_content='今天來介紹日本最具代表性的軟體開發手法\n它的名子為 **隕石落下型開發**。\n\n# 第一節\n\n通常的**瀑布式開發**是像下面這樣的形式:\n| 步驟 | 內容     | 負責人     |\n| ---- | -------- | ---------- |\n| 1    | 要件定義 | Producer   |\n| 2    | 基本設計 | Director   |\n| 3    | 詳細設計 | Planner    |\n| 4    | 實裝     | Programmer |\n\n\n而**隕石式開發**是像下面這樣子的形式：\n|     | 步驟 | 內容     | 負責人     |\n| --- | ---- | -------- | ---------- |\n| 神  | 1    | 要件定義 | Producer   |\n| 神  | 2    | 基本設計 | Director   |\n| 神  | 3    | 詳細設計 | Planner    |\n| 神  | 4    | 實裝     | Programmer |\n\n然後就會這樣（全部都被隕石砸到爆炸）：', metadata={'source': '..\\data\\notion\\隕石落下式開發法.md'}),
 Document(page_content='# 結論\n\n今天，我們介紹了日本的軟體開發現場偶爾自然地、應該說災厄地發生的隕石落下式開發法。\n\n**順道一提，對這種情況我們完全無法抵抗。** 你們，應該說我們，只能祈禱能儘早地去適應它。\n\n本文章為虛構，跟一切實際的人物、團體無關。（希望是這樣）', metadata={'source': '..\\data\\notion\\隕石落下式開發法.md'})]

# 4. QA over documents

In [11]:
def combine_documents(docs, document_separator="\n\n"):
    doc_strings = []
    for i, doc in enumerate(docs):
        doc_string = f"[文件{i+1}]\n\n{doc.page_content}"
        doc_strings.append(doc_string)
    return document_separator.join(doc_strings)
context = combine_documents(documents[:2])
print(context)

[文件1]

今天來介紹日本最具代表性的軟體開發手法
它的名子為 **隕石落下型開發**。

# 第一節

通常的**瀑布式開發**是像下面這樣的形式:
| 步驟 | 內容     | 負責人     |
| ---- | -------- | ---------- |
| 1    | 要件定義 | Producer   |
| 2    | 基本設計 | Director   |
| 3    | 詳細設計 | Planner    |
| 4    | 實裝     | Programmer |


而**隕石式開發**是像下面這樣子的形式：
|     | 步驟 | 內容     | 負責人     |
| --- | ---- | -------- | ---------- |
| 神  | 1    | 要件定義 | Producer   |
| 神  | 2    | 基本設計 | Director   |
| 神  | 3    | 詳細設計 | Planner    |
| 神  | 4    | 實裝     | Programmer |

然後就會這樣（全部都被隕石砸到爆炸）：

[文件2]

然後就會這樣（全部都被隕石砸到爆炸）：

💥要件定義💥Producer💥基本設計💥Director💥詳細設計💥Planner💥實裝💥Programmer

這是敏捷式開法守法的循環

[要件定義->基本設計->詳細設計->實裝]->[要件定義->基本設計->詳細設計->實裝]->

但在神的面前都是無力的（全部都被隕石砸到爆炸）

[要件💥定義-💥>基本設計💥->詳細設💥計->實💥裝]<->[要件定💥義->基本設💥計->詳💥細設💥計->💥實裝]->

在神的一聲令下全部都會**崩壞**，

而人民會努力地**重建**，

這就是 -- 隕石落下式開發法。

# 第二節

全部的計畫表都是由**天界**的一己之願來訂定，這被稱為**默示錄**

雖然在軟體工程中回饋是個重要的因素

但回饋**不會傳達**到神的耳裡

只是我們可以獻上我們的**祈禱**，這些祈禱在**非常稀有**的情況下能上達天聽。

神會以**各種姿態**出現，有從外部來的，也有從內部來的。

或者也可能是，還沒遇到的 or 見都見不到的某些人（像是版權持有人、專利所有者，這種形式的神明

In [12]:
template = """依照給定的資訊來回答下列的問題:
{context}

問題: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

# 5. 總結

In [13]:
# 建立一個全新的空的聊天紀錄
memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

In [14]:
# 首先我們先將 memory 中的聊天紀錄取出，並加入被接力的傳遞資料中
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
) # 記得 `RunnablePassThrough.assign(key=value)` 可視為 input_object[key] = value

# 根據使用者問題、聊天紀錄，我們想讓語言模型幫我們改寫成一個考慮到過去紀錄的獨立問題
standalone_question = {
    "standalone_question": {
        "question": itemgetter('question'),
        "chat_history": itemgetter('chat_history') | RunnableLambda(format_chat_history),
    }
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
}

# 根據這個獨立的問題，找出相關的文件
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}

# 將找到的文件和改寫後的使用者問題合成一個新的提示 (prompt)
final_inputs = {
    "context": lambda x: combine_documents(x["docs"]),
    "question": itemgetter("question"),
}

# 然後讓語言模型根據這些資訊產生回答
answer = {
    "answer": final_inputs | ANSWER_PROMPT | ChatOpenAI(temperature=0) | StrOutputParser(),
    "docs": itemgetter("docs"),
}

# 最後將所有串起來
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [15]:
with tracing_v2_enabled_if_api_key_set(project_name='tutorial'):
    inputs = {"question": "軟體開發中的神明是指什麼角色?"}
    outputs = final_chain.invoke(inputs)
    print("回答: ", outputs['answer']) # AIMessage
    print("參考文件: ", outputs['docs'])

<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>openresty</center>
</body>
</html>
).


回答:  在軟體開發中，神明的角色是指在建立秩序、制定計畫表、下達旨意等方面具有絕對權力的存在。他們以各種姿態出現，可能是從外部來的，也可能是從內部來的。他們可以是已經遇到的人，也可能是還沒遇到或見都見不到的人，如版權持有人、專利所有者等。然而，當有兩個以上的神明存在時，他們可能會下達相反的旨意，甚至吵架，這種情況被稱為諸神黃昏。此外，新的神明出現時，他們可能會將舊神一起吹飛，這被稱為聖戰。因此，在軟體開發中，神明的角色是具有絕對權力且可能產生衝突的存在。
參考文件:  [Document(page_content='神會以**各種姿態**出現，有從外部來的，也有從內部來的。\n\n或者也可能是，還沒遇到的 or 見都見不到的某些人（像是版權持有人、專利所有者，這種形式的神明，則更為凶惡）\n\n軟體開發的過程中，一步登天這樣有效率的方法是不存在，但相反的，像是天空之城裡一箭毀滅的因陀羅之箭卻是存在的。\n\n# 第三節\n\n但即使如此，由一個人的神明在建立的秩序**還算好的**。\n\n問題是，神明有**兩個人以上**的情況。這些神明們偶爾會下達相反的旨意，偶爾會吵架。\n\n這個我們稱為**諸神黃昏（Ragnarök）**（而受害的永遠都是我們這些人民們）\n\n或是，當原本的神明正在建築秩序時，新的神明出現，將舊神一起**全部吹飛**\n的事情也是有可能發生的。這個我們稱為**聖戰（Jihad）**\n\n根據不同的神明，也有擁有絕對的力量但是存在**極微稀薄**的神，而怎麼找也找不到的恐怕是邪神。\n\n然後我們千辛萬苦做出的供品，會在我們不知道的地方被**華麗地**發表。然後偶爾會從這邊再誕生出新的需求規格。\n\n# 結論\n\n今天，我們介紹了日本的軟體開發現場偶爾自然地、應該說災厄地發生的隕石落下式開發法。', metadata={'source': '..\\data\\notion\\隕石落下式開發法.md'}), Document(page_content='然後就會這樣（全部都被隕石砸到爆炸）：\n\n💥要件定義💥Producer💥基本設計💥Director💥詳細設計💥Planner💥實裝💥Programmer\n\n這是敏捷式開法守法的循環\n\n[要件定義->基本設計->詳細設計->實裝]->[要件定義->基本設計->詳細設計->實裝]

In [16]:
# Note that the memory does not save automatically
# This will be improved in the future
# For now you need to save it yourself
memory.save_context(inputs, outputs)
memory.load_memory_variables({}) # check

{'history': [HumanMessage(content='軟體開發中的神明是指什麼角色?'),
  AIMessage(content='在軟體開發中，神明的角色是指在建立秩序、制定計畫表、下達旨意等方面具有絕對權力的存在。他們以各種姿態出現，可能是從外部來的，也可能是從內部來的。他們可以是已經遇到的人，也可能是還沒遇到或見都見不到的人，如版權持有人、專利所有者等。然而，當有兩個以上的神明存在時，他們可能會下達相反的旨意，甚至吵架，這種情況被稱為諸神黃昏。此外，新的神明出現時，他們可能會將舊神一起吹飛，這被稱為聖戰。因此，在軟體開發中，神明的角色是具有絕對權力且可能產生衝突的存在。')]}

In [17]:
with tracing_v2_enabled_if_api_key_set(project_name='tutorial'):
    inputs2 = {"question": "那我們如何抵抗它?"}
    outputs2 = final_chain.invoke(inputs2)
    print("回答: ", outputs2['answer']) # AIMessage
    print("參考文件: ", outputs2['docs'])

回答:  在軟體開發中，應對具有絕對權力且可能產生衝突的神明角色，可以採取以下策略：

1. 建立良好的溝通與協調機制：確保不同神明之間能夠及時、有效地溝通，並協調彼此的旨意和需求。這可以透過定期的會議、討論和協商來實現。

2. 引入中立的調解者：在神明之間產生衝突時，引入一位中立的調解者或領導者，以幫助解決衝突並達成共識。這位調解者可以是項目經理或其他具有權威和中立性的人士。

3. 建立明確的優先順序和決策機制：確定不同神明的優先順序，並建立明確的決策機制，以便在出現衝突時能夠快速做出決策。這可以透過制定項目計劃、設立優先順序和指定負責人來實現。

4. 開放式的溝通和透明度：確保所有相關人員都能夠了解不同神明的旨意和需求，並能夠提供反饋和建議。這可以透過定期的進度報告、開放式的討論和透明的決策過程來實現。

5. 靈活適應和調整：在軟體開發過程中，可能會出現新的神明或需求，需要靈活適應和調整。這可以透過敏捷開發方法和迭代式開發來實現，以便及時應對變化和調整計劃。

總之，應對具有絕對權力且可能產生衝突的神明角色，需要建立良好的溝通、協調和決策機制，並保持開放的溝通和透明度，以便在軟體開發過程中能夠有效地處理衝突並達成共識。
參考文件:  [Document(page_content='神會以**各種姿態**出現，有從外部來的，也有從內部來的。\n\n或者也可能是，還沒遇到的 or 見都見不到的某些人（像是版權持有人、專利所有者，這種形式的神明，則更為凶惡）\n\n軟體開發的過程中，一步登天這樣有效率的方法是不存在，但相反的，像是天空之城裡一箭毀滅的因陀羅之箭卻是存在的。\n\n# 第三節\n\n但即使如此，由一個人的神明在建立的秩序**還算好的**。\n\n問題是，神明有**兩個人以上**的情況。這些神明們偶爾會下達相反的旨意，偶爾會吵架。\n\n這個我們稱為**諸神黃昏（Ragnarök）**（而受害的永遠都是我們這些人民們）\n\n或是，當原本的神明正在建築秩序時，新的神明出現，將舊神一起**全部吹飛**\n的事情也是有可能發生的。這個我們稱為**聖戰（Jihad）**\n\n根據不同的神明，也有擁有絕對的力量但是存在**極微稀薄**的神，而怎麼找也找不到的恐怕是邪神。\n\n然後我們千辛萬苦做出的供品，會在我們不知道的地方被**華麗地**發表。然後偶爾

如果單看第二次的問題「那我們如何抵抗它?」是不會知道在指什麼東西，但是根據過去的聊天紀錄，模型可以知道要抵抗的是神明。

下面我們實際看看它將問題改寫成什麼樣子，來融合過去聊天紀錄的資訊

In [18]:
(loaded_memory | standalone_question).invoke({"question": "那我們如何抵抗它?"})

{'standalone_question': '在軟體開發中，如何應對具有絕對權力且可能產生衝突的神明角色?'}